In [3]:
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
from bokeh.io import push_notebook
from ipywidgets import interact
from scipy.interpolate import Rbf
output_notebook()

Loading BokehJS ...

In [4]:
import pydgrid

ImportError: No module named 'pydgrid.pf'; 'pydgrid' is not a package

In [ ]:
sys1 = pydgrid.grid()
sys1.read('../examples/cigre/cigre_europe_residential.json')  # Load data
sys1.pf()  
sys1.get_v()      # post process voltages
sys1.get_i()      # post process currents

V_an  = np.array([item['v_an']/item['U_kV']/1000*np.sqrt(3) for item in sys1.buses])
V_bn  = np.array([item['v_bn']/item['U_kV']/1000*np.sqrt(3) for item in sys1.buses])
V_cn  = np.array([item['v_cn']/item['U_kV']/1000*np.sqrt(3) for item in sys1.buses])
Pos_x = np.array([item['pos_x'] for item in sys1.buses])
Pos_y = np.array([item['pos_y'] for item in sys1.buses])
x_min = np.min(Pos_x)
x_max = np.max(Pos_x)
y_min = np.min(Pos_y)
y_max = np.max(Pos_y)

x_margin = 150
y_margin = 100

x_min2 = x_min-x_margin
x_max2 = x_max+x_margin
y_min2 = y_min-y_margin
y_max2 = y_max+y_margin

Pos_x_e = np.hstack((x_min2, x_min2, Pos_x, x_max2, x_max2))
Pos_y_e = np.hstack((y_min2, y_max2, Pos_y, y_max2, y_min2))
V_an_e  = np.hstack((1,1, V_an, 1,1))
V_bn_e  = np.hstack((1,1, V_bn, 1,1))
V_cn_e  = np.hstack((1,1, V_cn, 1,1))
rbfi_a = Rbf(Pos_x_e, Pos_y_e, V_an_e, function='linear')
rbfi_b = Rbf(Pos_x_e, Pos_y_e, V_bn_e)
rbfi_c = Rbf(Pos_x_e, Pos_y_e, V_cn_e)

x =  np.linspace(x_min2,x_max2, 1000)
y =  np.linspace(y_min2,y_max2, 1000)
X, Y = np.meshgrid(x, y)
Z_a = rbfi_a(X, Y)
Z_b = rbfi_b(X, Y)
Z_c = rbfi_c(X, Y)

## CIGRE LV European System

### Graph with obtained results

In [ ]:
sys1.s_radio_scale = 0.1
sys1.bokeh_tools()

p = figure(width=600, height=800,
           title='European LV Network (CIGRE)', x_range=[-150,150], y_range=[-350, 50])

p.image(image=[Z_a], x=x_min2, y=y_min2, dw=(x_max2-x_min2), dh=(y_max2-y_min2), palette="Spectral11")


# lines:
source = ColumnDataSource(sys1.line_data)
lin = p.multi_line(source=source, xs='x_s', ys='y_s', color="red", alpha=1, line_width=5)

# buses:
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='x', y='y', size='s_radio', color='s_color', alpha=1)

p.add_tools(HoverTool(renderers=[lin], tooltips=sys1.line_tooltip))
p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))
show(p)

### Interaction with powers

In [ ]:
sys1.read('../examples/cigre/cigre_europe_residential.json')  # Load data
sys1.pf()  
sys1.get_v()      # post process voltages
sys1.get_i()      # post process currents

s_0_3pn = np.copy(sys1.pq_3pn) # 
s_0_1p  = np.copy(sys1.pq_1p)  # 
sys1.bokeh_tools()
p = figure(width=600, height=300,
           title='Voltage vs load powers', 
           x_range = [40,-350], y_range = [180,250],
           x_axis_label='Distance (m)',
           y_axis_label='Voltage (V)')
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='y', y='v_an', size=15, color="red", alpha=0.5)
p.circle(source=source, x='y', y='v_bn', size=15, color="green", alpha=0.5)
p.circle(source=source, x='y', y='v_cn', size=15, color="blue", alpha=0.5)
p.line([-400,300],[231*1.05,231*1.05], color='red', line_width=5)
p.line([-400,300],[231*0.90,231*0.90], color='blue', line_width=5)
#p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))


def update_loads(load_factor=1.0):

    #sys1.pq_1p = load_factor*s_0_1p
    sys1.pq_3pn = np.copy(load_factor*s_0_3pn)
    sys1.pf()
    sys1.get_v()
    sys1.get_i()
    sys1.bokeh_tools()

    source.data = sys1.bus_data
    push_notebook()

#p_grid = gridplot([[p], [p_2]])
show(p, notebook_handle=True)


In [ ]:
from ipywidgets import interact
interact(update_loads, load_factor=(-1,1.2, 0.01))

In [ ]:
np.sum(abs(sys1.pq_3pn), axis=1)

In [ ]:
sys1.params_pf['iters']

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()

In [ ]:
df['nodes'] = sys1.nodes
df['I_node_m'] = np.abs(sys1.I_node)
df['V_node_m'] = np.abs(sys1.V_node)
df['phi'] = np.angle(sys1.I_node, deg=True) - np.angle(sys1.V_node, deg=True)
df['s'] = np.conjugate(sys1.I_node)*sys1.V_node
df['p'] = df['s'].real
df['q'] = df['s'].imag

In [ ]:
self = sys1

node_sorter = []
for bus in self.buses:
    for node in range(10):
        bus_node = '{:s}.{:s}'.format(str(bus['bus']),str(node))
        if bus_node in self.nodes:
            node_idx = self.nodes.index(bus_node) 
            node_sorter += [node_idx]
                
                

In [ ]:
sys1.V_node[node_sorter]